# Analysis of the spatial structure of connectivity in the MICrONS dataset

The MICrONS initiative provided a dense reconstruction of around a cubic milimeter of mouse brain tissue.

At OBI, we have converted that data into the SONATA format that is often used to represent biophysically-detailed computational models of neuronal circuitry. We believe that this is a useful resource for the community for the following reasons:
 1. It allows direct comparison of models to the data, as both are in the same format. In the future it may even be possible to simulate the MICrONS circuitry as one simulates the computational models.
 2. There are many useful code libraries for analyzing SONATA-formatted circuits.
 3. It is reduced representation of the data. While this discards a lot of information, what remains is still very useful for many purposes. And the reduced data can be more easily handled and analyzed faster.
 4. During the conversion to SONATA we added derived data. Specifically, high-quality morphology skeletons with extracted spines.


Here, we want to expand on point (3) above. We demonstrate an examplary use case, where we analyze the data at the level of the wiring diagram, i.e., as a graph representation with neurons as vertices and the presence of a synaptic connection indicated by directed edges.

### Summary of the analysis

We will perform a proof-of-concept analysis that reveals how structured and non-random the connectivity is. Specifically, we will look at the spatial structure of connectivity, i.e., how it depends on the relative locations of pre- and post-synaptic somata. 

We will count how many connected pairs of neurons exist at given horizontal and vertical offsets and then look for _asymetries_. In distance-dependent, but unstructured connectivity, we would expect the same number of connections for, e.g., delta-x=100 um as for delta-x=-100 um. The exception would be the depth axis, as we know that connectivity is structured along cortical layers. But as we will see, the MICrONS data has structure even beyond that!

To our knowledge, this type of analysis has not been performed before, simply because no comparable dataset of biological connectivity was available. We argue that the MICrONS data has been a paradigm shift in terms of the quantity and density of available connectivity information, and the analyses we perform should reflect that and become more ambitious and diverse.

## Importing code libraries and loading the data

We import a number of standard packages, as well as _conntility_ and _connalysis_. These two packages provide (as we will see) useful functionality for the analysis of this type of data.

In [ ]:
import conntility
import pandas
import numpy

from matplotlib import pyplot as plt
from scipy.spatial.transform import Rotation

numpy.seterr(all="ignore")


fn = "../../../../shared_data/MICrONS_SONATA/microns_con_mat_multi.h5"

# We load the data that has been serialized into a single hdf5 file into an object.
M = conntility.ConnectivityMatrix.from_h5(fn)


### Side note: data representation

The data, that is, the neurons and their connections, are represented in the object M. 
The representation has a list of _vertices_, i.e. neurons, and _edges_, i.e. synaptic connections. 

We can list the vertices and their properties.
Important properties for this analysis are:
  - layer, a string indicating the cortical layer of each neuron
  - synapse_class, this is either "EXC" or "INH" indicating that a neuron is excitatory or inhibitory
  - x, y, z, the spatial locations of the neurons in um

In [ ]:
display(M.vertices)

We can also list the edges and their properties.
For the purpose of this analysis, we do not consider the properties at all - we are only interested in the presence or absence of a connection. Still, other analyses can use the properties.

For example, "spine_id" lists an identifier of the spine that a synapse innervates, or -1 for shaft synapses. Note that we have only identified spines for a subset of postsynaptic neurons, and for the rest all afferent synapses list a value of -1. We are working on extending the number of neurons with identified spines.

In [ ]:
display(M.edges.head())

## Select the excitatory sub-graph

It is quite accepted that connectivity of inhibitory neurons follows quite different rules than excitatory connectivity. 
Hence, we limit our analysis here to only the excitatory subgraph, for simplicity.

We also add a new property to all _vertices_: A representation of the neurons' layers, but integer valued. As a reminder, the existing "layer" property is represented as a string. 

In [ ]:
# Create a subcircuit using the .index functionality. The following creates the subcircuit of neurons where 
# the values of "synapse_class" is equal to "EXC".
M = M.index("synapse_class").eq("EXC")

# We add a new vertex property, using the existing "layer" property and converting it to integers.
M.add_vertex_property("int_layer", M.vertices["layer"].astype(str).astype(int).values)

### Example: Plotting neuron soma locations

As an example, we plot the soma locations of all neurons, using different colors for different layers.

We see that layers are separated mostly along the y-axis.

In [ ]:
ax = plt.figure().gca()

for l in [2, 3, 4, 5, 6]:  # All cortical layers except 1. Because 1 is almost all inhibitory.
    subM = M.index("int_layer").eq(l)
    ax.scatter(subM.vertices["x"], subM.vertices["y"], s=3, label=l)
    ax.set_xlabel("x (um)"); ax.set_ylabel("y (um)")
plt.legend()

ax = plt.figure().gca()

for l in [2, 3, 4, 5, 6]:  # All cortical layers except 1. Because 1 is almost all inhibitory.
    subM = M.index("int_layer").eq(l)
    ax.scatter(subM.vertices["x"], subM.vertices["z"], s=3, label=l)
    ax.set_xlabel("x (um)"); ax.set_ylabel("z (um)")
plt.legend()

## Rotating the volume

Here, we want to perform an analysis of connectivity with respect to the offset of connected pairs along three axes. 
We know that a vertical axis, i.e., orthogonal to layer boundaries is important for the structure of connectivity. 

Above we saw that this axis of organization is _mostly_ aligned with the y-axis, but not completely.
Hence, we rotate the volume such that it is completely aligned. 

After that operation, the y-axis is our "vertical" and conversely, x- and z-axis should be unaffected by the laminar structure of connectivity.

In [ ]:
axes = ["x", "y", "z"]

# We calculate the current "vertical" direction as follows:
# First, we obtain the mean x, y, z coordinates of somata associated with each layer.
per_layer_xyz = M.vertices.groupby("int_layer")[axes].mean()

# We make sure the data is sorted by layer from 2 to 6. 
# As we used the integer representation of layers, we can just use the regular sorting functionality.
per_layer_xyz = per_layer_xyz.sort_index()

# Calling "diff" on this will calculate a vector pointing from layer 2 to 3, from 3 to 4, etc.
# The average of all this is therefore a good approximation of the current vertical axis.
vertical = per_layer_xyz.diff(axis=0).mean().values
# Normalize
vertical = vertical / numpy.linalg.norm(vertical)

# Next, we calculate the rotation that turns the vertical vector into [0, 1, 0], i.e., the y-axis
rot, _ = Rotation.align_vectors(numpy.array([0, 1, 0]), vertical)

# As we will see: vertical is already quite close to being aligned with the y-axis. 
print(vertical)
print(rot.as_matrix())

# Perform the rotation. 
# For the names of the rotated coordinates we use the upper-case "X", "Y" and "Z"
xyz_out = rot.apply(M.vertices[axes].values)
for _col, _vals in zip(axes, xyz_out.transpose()):
    M.add_vertex_property(_col.upper(), _vals)


For validation, we repeat the previous plot, but this time using the rotated coordinates

In [ ]:
ax = plt.figure().gca()

for l in [2, 3, 4, 5, 6]:  # All cortical layers except 1. Because 1 is almost all inhibitory.
    subM = M.index("int_layer").eq(l)
    ax.scatter(subM.vertices["X"], subM.vertices["Y"], s=3, label=l)
    ax.set_xlabel("X (um)"); ax.set_ylabel("Y (um)")
plt.legend()

ax = plt.figure().gca()

for l in [2, 3, 4, 5, 6]:  # All cortical layers except 1. Because 1 is almost all inhibitory.
    subM = M.index("int_layer").eq(l)
    ax.scatter(subM.vertices["X"], subM.vertices["Z"], s=3, label=l)
    ax.set_xlabel("X (um)"); ax.set_ylabel("Z (um)")
plt.legend()

## Creating a random control

The results of connectivity analyses can be hard to interpret. It often helps to compare to a random control.

In our example, we want to argue that the connectivity of MICrONS has structure that is not expected to emerge in randon connectivity. So we built a random control that only captures the following aspects:
  - Connectivity in the random control will be distance-dependent.
  - Connectivity in the random control will be structured by cortical layers. This is done by using different parameters for the distance-dependent connectivity for each combination of pre- and post-synaptic layer.

Generating such a control takes around 3 minutes. To speed up this demo, we have already prepared a control instance and simply load it, instead of generating a new one here.

In [ ]:
fn_ctrl = "../../../../shared_data/MICrONS_SONATA/control_con_mat.h5"
C = conntility.ConnectivityMatrix.from_h5(fn_ctrl)


## Extracting the relevant information from data and control

As mentioned, we are interested in the offsets along the axes of connected neuron pairs and how many there are.

conntility provides a relatively simple way of calculating this.

The function "edge_associated_vertex_properties" returns the values of a specified vertex property for the pre- and post-synaptic neurons of each connection. 

Here is an example where we ask for the "int_layer" property values and use it to rapidly get the structural strengths of all laminar pathways.

In [ ]:
layer_df = M.edge_associated_vertex_properties("int_layer")

# The returned DataFrame has one row per connection and two columns. 
# The column called "row" has the data for the pre-synaptic neuron; "col" for the post-synaptic neuron.
# (At OBI, we strongly believe that the competing standard where the row of a connectivity matrix indicates
# the post-synaptic neuron is wrong.)
display(layer_df)

# We can count the values to get (laminar) pathway strengths
layer_df.value_counts().unstack("col")

If we use this functionality for the "X", "Y" and "Z" properties, we can then subtract the values in the first column from the values in the second column. The result is, for each connection, a vector pointing from the pre- to the post-synaptic neuron.

In [ ]:
def build_edge_delta_dataframe(M):
    # Get the pre- and post-synaptic X-coordinates for all connections
    edge_x = M.edge_associated_vertex_properties("X")
    # Subtract values for pre- from values for post-synaptic neurons
    dx = edge_x["col"] - edge_x["row"]
    # Same for Y and Z
    edge_y = M.edge_associated_vertex_properties("Y")
    dy = edge_y["col"] - edge_y["row"]
    edge_z = M.edge_associated_vertex_properties("Z")
    dz = edge_z["col"] - edge_z["row"]

    dx.name = "X"; dy.name = "Y"; dz.name= "Z"
    
    # We also attach the values of pre- and post-synaptic layer
    # Later on, we use this to filter the data in order to analyze specific pathways.
    edge_pw = M.edge_associated_vertex_properties("layer").rename(columns={"row": "source_layer", "col": "target_layer"})

    deltas = pandas.concat([
        dx, dy, dz, edge_pw
    ], axis=1)
    return deltas

# Assemble the DataFrames for both MICrONS data...
deltas = build_edge_delta_dataframe(M)
# ... and control
deltas_ctrl = build_edge_delta_dataframe(C)
display(deltas.head())

### Counting the number of connected pairs at given offsets

We can use these DataFrames to quickly count and visualize the numbers of connected pairs at given offsets.

To do this, we define a helper function that digitizes (bins) the data into spatial bins with a specifiable resolution.

Then, we can once again use the "value_counts" functionality to count the numbers of pairs in each bin and visualize the result.

In [ ]:
def digitized(df_in, resolutions):
    """
    df_in: The input dataframe holding the spatial connectivity data.
    resolutions: A pandas.Series specifying the resolution requested. The data for each
      axis named in the index of the Series will be binned with the associated resolution in um.
      See below for an example.
    """
    return (df_in[resolutions.index] / resolutions).round() * resolutions

# Helper function to generate an image plot
def count_dataframe_to_image(I, **kwargs):
    extent = [I.columns[0], I.columns[-1], I.index[0], I.index[-1]]
    ax = plt.figure().gca()
    plt.imshow(I, extent=extent, **kwargs)
    
    ax.set_frame_on(False)
    ax.set_xlim(ax.get_xlim())
    ax.set_ylim(sorted(ax.get_ylim())[::-1])
    ax.plot(ax.get_xlim(), [0, 0], color="black", lw=0.5)
    ax.plot([0, 0], ax.get_ylim(), color="black", lw=0.5)
    return ax

# Digitize "X" and "Z" with 20 um each
resolutions = pandas.Series({"X": 20.0, "Z": 20.0})
dgtz_deltas = digitized(deltas, resolutions)

# Run value_counts and visualize
I = dgtz_deltas.value_counts().sort_index().unstack("Z", fill_value=0)
ax = count_dataframe_to_image(I)
ax.set_xlabel("dZ (um)")
ax.set_ylabel("dX (um)")

### Visualizing asymmetry

The result above looks not very interesting at all. Most connections are at low distances and they fall off similarly in all (horizontal) directions.

However, there is structure in there that we can find by considering specifically _asymmetry_:
Consider a connection at, e.g., dX=200 um, dZ=-100 um. For that pair, it is possible that a connection also exists in the opposite direction, i.e. they are reciprocally connected. That connection would show up at dX=-200 um, dZ=100 um.

In unstructured connectivity, the probability that a connection exists in one direction is equal to the probability for the other direction. Consequently, the number of pairs in dX=200, dZ=-100 should be equal to the number of pairs in dX=-200, dZ=100.

We can test this by calculating the (normalized) difference between the data and its spatial transpose.

In [ ]:
def spatial_asymmetry(df_fw, df_bw, axes_to_use, min_count=50):
    """
    Calculates the spatial asymmetry of relative offsets of connected pairs.
    df_in: DataFrame holding the spatial connectivity data.
    axes_to_use: The names of the two spatial axes to consider.

    Note: The data in the axes_to_use columns must have been already binned using the "digitized" function!
    """
    # Regular counts for each spatial bin.
    # Note: axes_to_use[0] has no specific meaning here. We could use any column name, as we are only considering the count.
    A = df_fw.groupby(axes_to_use)[axes_to_use[0]].count()
    # Counts in the bins representing the corresponding reverse connections.
    B = df_bw.groupby(axes_to_use)[axes_to_use[0]].count()

    # Difference in counts in the spatial bins.
    abDiff = A.subtract(B, fill_value=0)
    # Calculate also the sum. For normalization.
    abSum = A.add(B, fill_value=0)

    # Normalized difference. Note: 
    # A resulting value of 1 indicates connection only in the forward direction.
    # 0 indicates equal number forward and backward.
    # -1 only backward direction.
    I = abDiff / abSum

    # Where the number of samples is insufficient, we set to nan.
    I[abSum < min_count] = numpy.nan
    return I

# We can get the spatial bin of the corresponding backward connection simply by 
# multiplying the delta-x/y/z with -1.
I = spatial_asymmetry(dgtz_deltas, -dgtz_deltas, ["X", "Z"]).unstack("Z")
ax = count_dataframe_to_image(I, cmap="coolwarm", clim=[-1, 1])
plt.colorbar(label="Asymmetry")
ax.set_xlabel("dZ (um)")
_ = ax.set_ylabel("dX (um)")


We see that for the local range (up to 200 um) there is no asymmetry.

But for more long-range connections, we find a strong asymmetry along the x-axis.

**NOTE**: The MICrONS data represents a volume along the border between the primary visual area and a higher visual area. The areas are separate mostly along the x-axis. Hence, we believe that this asymmetry may be related to long-range, inter-regional connections.

## Comparing to the control and more

To assess significance, we re-create the same analysis in an interactive widget that allows us to swap between MICrONS data and control. It also allows us to adjust the spatial resolution and which spatial axes are considered.

This allows us to also consider asymmetry along the y-axis. While we expect such an asymmetry due to the laminar structure of connectivity, we will see that it is more complex than expected. For example, the asymmetry along the y-axis depends also on the offset along the x-axis. This demonstrates different laminar rules for local vs. long-range connectivity.

Swap the data source for "control" to see how significant the results are.

**Interpretation help**: The y-axis indicates "depth". Hence, a positive value of dY indicates a _downward_ connection.

In [ ]:
from ipywidgets import widgets

wgt_res = widgets.FloatSlider(value=15.0, min=5.0, max=100.0, description="Resolution")
axis_hor = widgets.Select(options=["X", "Y", "Z"], value="Z", description="Hor. axis")
axis_vert = widgets.Select(options=["X", "Y", "Z"], value="X", description="Vert. axis")
wgt_data = widgets.Dropdown(options=[("MICrONS", 0), ("Control", 1)], index=0, description="Data source")

def display_func_smpl(res, data_idx, ax_hor, ax_vert):
    # Select data source. 0 = MICrONS. 1 = Control
    data = [deltas, deltas_ctrl][data_idx]
    dgtz_data = digitized(data,
                pandas.Series({"X": res, "Y": res, "Z": res}))
    I = spatial_asymmetry(dgtz_data, -dgtz_data, [ax_hor, ax_vert]).unstack(ax_hor)
    ax = count_dataframe_to_image(I, clim=[-0.5, 0.5], cmap="coolwarm")
    plt.colorbar(label="Asymmetry")
    ax.set_xlabel("d" + ax_hor + " (um)")
    ax.set_ylabel("d" + ax_vert + " (um)")
    
i = widgets.interactive(display_func_smpl,
                        data_idx=wgt_data,
                        res=wgt_res,
                        ax_hor=axis_hor,
                        ax_vert=axis_vert)
display(i)

Below, we provide a more complex version of that widget that additionally allows:

 - Apply a rotation around whichever axis has been specified as the vertical axis before analysis.
 - Select specific pathways in terms of source and target layers.

**Note**: When you visualize asymmetry along the y-axis and the source and target layers are different, the result is tricky to interpret. Because, by definition the pathway from a superficial to a deeper layer can only have downward connections.


Play around with the widget to explore the amazing complexity of biological connectivity!

In [ ]:
from ipywidgets import widgets

def filtered(df_in, fltr_dict):
    """
    Filter a DataFrame according to values in specified columns.
    """
    for k, v in fltr_dict.items():
        df_in = df_in.loc[df_in[k].isin(v)]
    return df_in.copy()

def rotated(df_in, cols, angle):
    """
    Perform custom spatial rotation for data in a DataFrame.
    """
    assert len(cols) == 2
    v_in = df_in[cols].values  # n x 2
    mat = numpy.array([
        [numpy.cos(angle), -numpy.sin(angle)],
        [numpy.sin(angle), numpy.cos(angle)]
        ])
    v_out = numpy.dot(v_in, mat)
    df_out = pandas.concat(
        [
            pandas.DataFrame(v_out, columns=cols, index=df_in.index),
            df_in.drop(columns=cols)
        ], axis=1
    )
    return df_out

wgt_src_l = widgets.SelectMultiple(options=deltas.source_layer.drop_duplicates().astype(str).sort_values(),
                                   index=tuple(range(len(deltas.source_layer.drop_duplicates()))),
                                   description="Source layers")
wgt_tgt_l = widgets.SelectMultiple(options=deltas.target_layer.drop_duplicates().astype(str).sort_values(),
                                   index=tuple(range(len(deltas.target_layer.drop_duplicates()))),
                                   description="Target layers")
wgt_rot = widgets.FloatSlider(value=0.0, min=-3.15, max=3.15, step=0.05, description="Rotation")
wgt_data = widgets.Dropdown(options=[("MICrONS", 0), ("Control", 1)], index=0, description="Data source")

def display_func_cmplx(res, rot, data_idx, src_l, tgt_l, ax_hor, ax_vert):
    # Select data source
    data = [deltas, deltas_ctrl][data_idx]
    # For the rotation. Rotation is around "ax_vert", hence its coordinates will be unaffected and it is removed.
    all_axes = ["X", "Y", "Z"]
    all_axes.remove(ax_vert)

    # Apply pathway filter. NOTE: Since we have separate source and target layers, we 
    # have to swap their roles for the corresponding backward data.
    df_fw = filtered(data, {"source_layer": src_l, "target_layer": tgt_l})
    df_bw = filtered(data, {"source_layer": tgt_l, "target_layer": src_l})
    # Apply rotation
    df_fw = rotated(df_fw, all_axes, rot)
    df_bw = rotated(df_bw, all_axes, rot)
    # Binning. Also multiply the backward data with -1 here.
    df_fw = digitized(df_fw, pandas.Series({ax_hor: res, ax_vert: res}))
    df_bw = -digitized(df_bw, pandas.Series({ax_hor: res, ax_vert: res}))

    # Calculate, then visualize
    I = spatial_asymmetry(df_fw, df_bw, [ax_hor, ax_vert]).unstack(ax_hor)
    ax = count_dataframe_to_image(I, clim=[-0.5, 0.5], cmap="coolwarm")
    plt.colorbar(label="Asymmetry")
    ax.set_xlabel("d" + ax_hor + " (um)")
    ax.set_ylabel("d" + ax_vert + " (um)")
    
i = widgets.interactive(display_func_cmplx,
                        data_idx=wgt_data,
                        src_l=wgt_src_l,
                        tgt_l=wgt_tgt_l,
                        res=wgt_res,
                        ax_hor=axis_hor,
                        ax_vert=axis_vert,
                        rot=wgt_rot)
display(i)

## A related analysis of asymmetry

Finally, we provide one last analysis. It is conceptually similar, but slightly different than the one above.

The previous analysis only ever considered two of the spatial axes at a time. Here, all three are considered.

For a given (binned) spatial offset, e.g., dX=200, dZ=250, we consider the location of targeted neurons along the remaining axis. We then ask whether the values are above or below the overall mean of the chosen pathway.

For example, if we select X and Z as horizontal and vertical axes, then red color indicates that the Y-locations of targeted neurons in a spatial bin are larger than the mean over all spatial bins. This would indicate the deeper neurons are targeted.

This analysis is - admittedly - hard to interpret. Hence it is important to compare to the control. But once again, we find substantial stucture that is not present in the control.

In [ ]:
df_absolute_locations = pandas.concat(
    [
        M.edge_associated_vertex_properties("X").rename(
            columns={
                "row": "src_X", "col": "tgt_X"
            }
        ),
        M.edge_associated_vertex_properties("Y").rename(
            columns={
                "row": "src_Y", "col": "tgt_Y"
            }
        ),
        M.edge_associated_vertex_properties("Z").rename(
            columns={
                "row": "src_Z", "col": "tgt_Z"
            }
        )
    ], axis=1
)
deltas_and_abs = pandas.concat([deltas, df_absolute_locations], axis=1)

df_absolute_locations_ctrl = pandas.concat(
    [
        C.edge_associated_vertex_properties("X").rename(
            columns={
                "row": "src_X", "col": "tgt_X"
            }
        ),
        C.edge_associated_vertex_properties("Y").rename(
            columns={
                "row": "src_Y", "col": "tgt_Y"
            }
        ),
        C.edge_associated_vertex_properties("Z").rename(
            columns={
                "row": "src_Z", "col": "tgt_Z"
            }
        )
    ], axis=1
)
deltas_and_abs_ctrl = pandas.concat([deltas_ctrl, df_absolute_locations_ctrl], axis=1)

In [ ]:
from ipywidgets import widgets

def centered(df_in, cols_to_center):
    for col in cols_to_center:
        df_in[col] = df_in[col] - df_in[col].mean()
    return df_in

def asymmetry_counter(series_in, min_count=25):
    if len(series_in) < min_count: return numpy.nan
    return series_in.mean()
    # return ((series_in > 0).sum() - (series_in < 0).sum()) / len(series_in)  # Alternative calculation


def display_func1(res, data_idx, side, src_l, tgt_l, ax_hor, ax_vert):
    data = [deltas_and_abs, deltas_and_abs_ctrl][data_idx]
    ax_asym = [_ax for _ax in ["X", "Y", "Z"] if _ax not in [ax_hor, ax_vert]][0]
    ax_asym = side + "_" + ax_asym

    df = filtered(data, {"source_layer": src_l, "target_layer": tgt_l})
    df = centered(df, [ax_asym])
    df = pandas.concat([digitized(df, pandas.Series({ax_hor: res, ax_vert: res})), df[ax_asym]], axis=1)
    I = df.groupby([ax_vert, ax_hor])[ax_asym].apply(asymmetry_counter)
    I = I.sort_index().unstack(ax_hor)

    ax = count_dataframe_to_image(I, cmap="coolwarm", clim=[-100, 100])
    plt.colorbar()

    ax.set_xlabel("d" + ax_hor + " (um)")
    ax.set_ylabel("d" + ax_vert + " (um)")


# We re-create all the widgets. Otherwise, whenever we change something here, also the plots above are updated.
wgt_src_l = widgets.SelectMultiple(options=deltas.source_layer.drop_duplicates().astype(str).sort_values(),
                                   index=tuple(range(len(deltas.source_layer.drop_duplicates()))),
                                   description="Source layers")
wgt_tgt_l = widgets.SelectMultiple(options=deltas.target_layer.drop_duplicates().astype(str).sort_values(),
                                   index=tuple(range(len(deltas.target_layer.drop_duplicates()))),
                                   description="Target layers")
wgt_res = widgets.FloatSlider(value=15.0, min=5.0, max=100.0, description="Resolution")
axis_hor = widgets.Select(options=["X", "Y", "Z"], value="Z", description="Hor. axis")
axis_vert = widgets.Select(options=["X", "Y", "Z"], value="X", description="Vert. axis")
wgt_data = widgets.Dropdown(options=[("MICrONS", 0), ("Control", 1)], index=0, description="Data source")
wgt_side = widgets.Dropdown(options=[("Source", "src"), ("Target", "tgt")], index=1, description="Side")

i = widgets.interactive(display_func1,
                        res=wgt_res,
                        data_idx=wgt_data,
                        src_l=wgt_src_l,
                        tgt_l=wgt_tgt_l,
                        ax_hor=axis_hor,
                        ax_vert=axis_vert,
                        side=wgt_side
                        )
display(i)